In [ ]:
!pip install --upgrade transformers datasets evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from datasets import Dataset, DatasetDict, ClassLabel
from google.colab import drive
import os
import evaluate
# Mount Google Drive
drive.mount('/content/drive')

# ─── Configuration ────────────────────────────────────────────────────────────
MODEL_CHECKPOINT = "roberta-base"  # public alias for Facebook's RoBERTa
EMOTIONS        = ["anger", "disgust", "fear", "joy", "neutral", "sadness", "surprise"]
NUM_LABELS      = len(EMOTIONS)
MAX_LEN         = 128
BATCH_SIZE      = 16
EPOCHS          = 3
LR              = 2e-5

# ─── 1) Load CSVs into pandas ─────────────────────────────────────────────────
df_train = pd.read_csv("/content/drive/MyDrive/MELD/train_with_context.csv")
df_dev   = pd.read_csv("/content/drive/MyDrive/MELD/dev_with_context.csv")
df_test  = pd.read_csv("/content/drive/MyDrive/MELD/test_with_context.csv")

# ─── 2) Convert pandas DataFrames to Hugging Face Datasets ────────────────────
train_ds = Dataset.from_pandas(df_train)
dev_ds   = Dataset.from_pandas(df_dev)
test_ds  = Dataset.from_pandas(df_test)

# Remove the pandas index column if present
for ds in (train_ds, dev_ds, test_ds):
    if "_pandas_index" in ds.column_names:
        ds = ds.remove_columns(["_pandas_index"])

dataset = DatasetDict({"train": train_ds, "validation": dev_ds, "test": test_ds})

# ─── 3) Encode labels as ClassLabel ───────────────────────────────────────────
label_feature = ClassLabel(names=EMOTIONS)

def add_labels(example):
    example["labels"] = label_feature.str2int(example["Emotion"])
    return example

dataset = dataset.map(add_labels)

# ─── 4) Tokenize bert_input column ────────────────────────────────────────────
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

def tokenize_batch(examples):
    return tokenizer(
        examples["bert_input"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN
    )

dataset = dataset.map(tokenize_batch, batched=True)

# ─── 5) Prepare for PyTorch ───────────────────────────────────────────────────
dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"]
)

# ─── 6) Load the RoBERTa sequence classification model ────────────────────────
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_CHECKPOINT,
    num_labels=NUM_LABELS
)

# ─── 7) Define evaluation metric ──────────────────────────────────────────────
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return metric.compute(predictions=preds, references=labels)

# ─── 8) Set up training arguments ──────────────────────────────────────────────
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/MELD/teacher_roberta_erc",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

# ─── 9) Initialize Trainer and fine-tune ─────────────────────────────────────
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.save_model("/content/drive/MyDrive/MELD/teacher_roberta_erc")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Map:   0%|          | 0/9988 [00:00<?, ? examples/s]

Map:   0%|          | 0/1108 [00:00<?, ? examples/s]

Map:   0%|          | 0/2610 [00:00<?, ? examples/s]

Map:   0%|          | 0/9988 [00:00<?, ? examples/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: konthodores (konthodores-university-of-patras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,1.244600,1.201751,0.612816
2,1.030200,1.139270,0.629061
3,0.901000,1.133685,0.620036


In [ ]:
from transformers import TrainingArguments
import inspect

# Print the signature
print(inspect.signature(TrainingArguments.__init__))

# Or get full help text
help(TrainingArguments)


(self, output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: Optional[float] = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_steps: int = -1, lr_scheduler_type: Union[transformers.trainer_utils.SchedulerType, str] = 'linear', lr_scheduler_kwargs: Union[dict, str, NoneType] = <factory>, warmup_ratio: float = 0.0, warmup_ste